# KHỞI TẠO THƯ VIỆN CẦN THIẾT

In [2]:
import pandas as pd
import yaml
import pyodbc
from sqlalchemy import create_engine
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash
from datetime import timedelta

import sys
sys.path.append(r"G:/My Drive/HỌC TẬP DUE/NNT_năm 4 kì 1/DW & DM/config")
import mdl_db_connection

# KHỞI TẠO HÀM

convert csv file to df

tạo def

In [3]:
def check_common_columns(df_1,df_2):
    #get columns name to lisst
    column_names_df_1 = df_1.columns.tolist()
    column_names_df_2 = df_2.columns.tolist()
    
    #convert list to set
    set_column_names_df_1 = set(column_names_df_1)
    set_column_names_df_2 = set(column_names_df_2)

    #check common columns
    common_columns = set_column_names_df_1.intersection(set_column_names_df_2)
    print(f"Các cột chung trong hai danh sách và là:", common_columns)

def merge_df(df_1,df_2,key_column,how,name=None):
    check_common_columns(df_1, df_2)
    if name:
        print("DataFrame được gọi: ", name)
    df = pd.merge(df_1, df_2, on=key_column, how=how)

    print(f"""Dataframe 1 có (hàng, cột): {df_1.shape}\nDataframe 2 có (hàng, cột): {df_2.shape}\nNối bằng cột: {key_column}\nCách nối: {how}\nKết quả sau khi merge (hàng, cột): {df.shape}\n""")

    print("=" * 30)
    return df


# TẠO ETL

## EXTRACT

In [4]:
@task(name="convert sheet to dataframe", 
      description="có nhiều file csv nên mình ném vào dics return rồi lấy ra sau",
      task_run_name="convert csv to dataframe",
      cache_key_fn=task_input_hash, 
      cache_expiration=timedelta(days=1))
def extract_csv():
    dataframes = {}  # Khởi tạo một từ điển để lưu trữ các DataFrame
    dataframes['categories'] = pd.read_csv("categories.csv", sep=';')
    dataframes['customers'] = pd.read_csv("customers.csv")
    dataframes['employee_territories'] = pd.read_csv("employee_territories.csv")
    dataframes['employees'] = pd.read_csv("employees.csv")
    dataframes['order_details'] = pd.read_csv("order_details.csv")
    dataframes['orders'] = pd.read_csv("orders.csv")
    dataframes['products'] = pd.read_csv("products.csv")
    dataframes['regions'] = pd.read_csv("regions.csv")
    dataframes['shippers'] = pd.read_csv("shippers.csv")
    dataframes['suppliers'] = pd.read_csv("suppliers.csv")
    dataframes['territories'] = pd.read_csv("territories.csv")
    return dataframes  # Trả về từ điển chứa tất cả các DataFrame



## TRANSFORM

In [5]:
@task(name="merge join outer df", 
      description="yêu cầu là nối lại với nhau để tạo bảng lớn, xóa đi các record với orderID là null",
      task_run_name="merge join df",
      cache_key_fn=task_input_hash, 
      cache_expiration=timedelta(days=1)
    )
def transform_df(dfs):
  logger = get_run_logger()
  logger.info("dùng logger - transform data")
  print("print - transform data")

  #gọi ra các df trong dfs
  df_categoires = dfs['categories']
  df_customers = dfs['customers']
  df_employee_territories = dfs['employee_territories']
  df_employees = dfs['employees']
  df_order_details = dfs['order_details']
  df_orders = dfs['orders']
  df_products = dfs['products']
  df_regions = dfs['regions']
  df_shippers = dfs['shippers']
  df_suppliers = dfs['suppliers']
  df_territories = dfs['territories']

  df_1 = merge_df(df_categoires, df_products,'categoryID','outer',"df_categoires và df_products")
  df_2 = merge_df(df_1, df_suppliers, 'supplierID', 'outer', "df_cate_prd và df_suppliers")
  df_3 = merge_df(df_2, df_order_details, 'productID', 'outer' )
  df_4 = merge_df(df_3, df_orders, 'orderID', 'outer','df_cate_prd_supp_order_details và df_orders')

  df_5 = merge_df(df_4, df_customers, 'customerID', 'left')

  df_6 = merge_df(df_5, df_employees, 'employeeID', 'left')

  df_7 = merge_df(df_6, df_employee_territories, 'employeeID', 'left')

  df_8 = merge_df(df_7, df_territories, 'territoryID', 'left')


  df_final = merge_df(df_8, df_regions, 'regionID', 'left')

  df_final = df_final.dropna(subset=['orderID'])


  return df_final

## LOAD

In [6]:
@task(name="load df vào database", 
      description="lưu vào db để sau sử dụng lại", 
      task_run_name="load df vào database",
      cache_key_fn=task_input_hash, #lưu lại cache
      cache_expiration=timedelta(days=1) #thời gian cache lưu lại
    )
def load_df_to_db(df_final):
    logger = get_run_logger()
    logger.info("dùng logger - load df vào database")
    print("print - load df vào database")
    conn_str, engine = mdl_db_connection.connect_db()
    conx = pyodbc.connect(conn_str) 
    # Đổi 'table_name' thành tên bảng bạn muốn lưu dữ liệu vào
    table_name = 'ETL_CSV_02092023'
    # Lưu DataFrame vào cơ sở dữ liệu
    df_final.to_sql(table_name, con=engine, if_exists='replace', index=False)
    conx.commit()
    # Đóng kết nối pyodbc
    conx.close()


## BUILD FLOW

In [7]:
@flow(
      name = "FLOW ETL csv TO DB - WEEK 3 - USING PREFECT PYTHON"
      , flow_run_name="ETL csv to db"
      # , retries=1
      # , retry_delay_seconds=5
      , log_prints=True
      , description="load hết mấy file csv nối lại và ném vào db"
      )
def build_flow(schedule=None):
    # hiển thị thông tin flow chạy
    logger = get_run_logger()
    logger.info("ETL FILE CSV TO DATABASE USING PREFECT + PYTHON - 62_Nguyen Ngo Thuong SV_ETL_CSV ")
    # print("print - 62_Nguyen Ngo Thuong SV_ETL_CSV")

    dfs = extract_csv()
    df_final = transform_df(dfs)
    load_df_to_db(df_final)

    return flow

In [8]:
# Đặt biến table_name ở phạm vi toàn cục
table_name = 'ETL_CSV_02092023'

flow = build_flow()


00:47:22.381 | INFO    | prefect.engine - Created flow run 'apricot-frigatebird' for flow 'FLOW ETL csv TO DB - WEEK 3 - USING PREFECT PYTHON'

00:47:22.542 | INFO    | Flow run 'ETL csv to db' - ETL FILE CSV TO DATABASE USING PREFECT + PYTHON - 62_Nguyen Ngo Thuong SV_ETL_CSV

00:47:22.586 | INFO    | Flow run 'ETL csv to db' - Created task run 'convert sheet to dataframe-0' for task 'convert sheet to dataframe'

00:47:22.588 | INFO    | Flow run 'ETL csv to db' - Executing 'convert sheet to dataframe-0' immediately...

00:47:22.661 | INFO    | Task run 'convert sheet to dataframe-0' - Finished in state Cached(type=COMPLETED)

...

00:47:22.849 | INFO    | Task run 'merge join outer df-0' - Finished in state Cached(type=COMPLETED)

00:47:22.988 | INFO    | Flow run 'ETL csv to db' - Created task run 'load df vào database-0' for task 'load df vào database'

00:47:22.990 | INFO    | Flow run 'ETL csv to db' - Executing 'load df vào database-0' immediately...

00:47:23.086 | INFO    | Task run 'load df vào database-0' - Finished in state Cached(type=COMPLETED)

00:47:23.126 | INFO    | Flow run 'apricot-frigatebird' - Finished in state Completed()

Kiểm tra kết nối, truy vấn thử

In [10]:
conn_str, engine = mdl_db_connection.connect_db()

# Kết nối đến cơ sở dữ liệu
conx = pyodbc.connect(conn_str)

# Truy vấn để lấy dữ liệu
query = f"SELECT * FROM {table_name}"

# Đọc dữ liệu từ truy vấn và tạo DataFrame
data = pd.read_sql(query, conx)

# Đóng kết nối pyodbc
conx.close()

# Hiển thị DataFrame
print(data.head)


c:\Users\admin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<bound method NDFrame.head of        categoryID  categoryName                              description  \
0               1     Beverages  Soft drinks coffees teas beers and ales   
1               1     Beverages  Soft drinks coffees teas beers and ales   
2               6  Meat/Poultry                           Prepared meats   
3               6  Meat/Poultry                           Prepared meats   
4               8       Seafood                         Seaweed and fish   
...           ...           ...                                      ...   
10124           8       Seafood                         Seaweed and fish   
10125           8       Seafood                         Seaweed and fish   
10126           8       Seafood                         Seaweed and fish   
10127           8       Seafood                         Seaweed and fish   
10128           8       Seafood                         Seaweed and fish   

       productID       productName  supplierID     quanti

# test prefect